In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2

import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

cudnn.benchmark = True
plt.ion()   # interactive mode

load from csv file images and labels, split them into a train/validation data file

In [13]:
img_table = pd.read_csv('/data/VPS/VPS_03/lmq/pancreas/pancr_net.csv')
X = np.array(img_table['Path'])
Y = np.array(img_table['var_0'])
x_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
print(len(X))
train_df = pd.DataFrame({'Path':x_train, 'var':y_train})
val_df = pd.DataFrame({'Path':X_test, 'var':Y_test})


26061


In [6]:
import albumentations

IMAGENET_SIZE = 224

train_transform = albumentations.Compose([
        
    albumentations.Resize(IMAGENET_SIZE, IMAGENET_SIZE),
    # albumentations.RandomCrop(512, 512),
        # illumilation
    # albumentations.JpegCompression(quality_lower=99, quality_upper=100,p=0.5),
    albumentations.OneOf([
        albumentations.RandomGamma(gamma_limit=(60, 120), p=0.9),
        albumentations.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.9),
        albumentations.RandomBrightness(limit=0.2, p=0.9),
        albumentations.RandomContrast(limit=0.2, p=0.9)
        ]),
#                                    CLAHE(clip_limit=4.0, tile_grid_size=(3, 3), p=1)
    # albumentations.GaussNoise(var_limit=(10, 30), p=0.5),
    
    albumentations.HorizontalFlip(p=0.5),
    albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, interpolation=cv2.INTER_LINEAR,border_mode=cv2.BORDER_CONSTANT, p=1),
#                                        OpticalDistortion(distort_limit=0.05, shift_limit=0.05, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_REFLECT_101, p=1)
    albumentations.OneOf([
        albumentations.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
        albumentations.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1)
    ], p=0.5),
    albumentations.Normalize(mean=(0.485, 0.12, 0.005), std=(0.081, 0.32, 0.043), max_pixel_value=255.0, p=1.0),
])

val_transform = albumentations.Compose([
    albumentations.Resize(IMAGENET_SIZE, IMAGENET_SIZE),
    # albumentations.CenterCrop(512, 512),
    # albumentations.JpegCompression(quality_lower=99, quality_upper=100,p=1),
    albumentations.Normalize(mean=(0.485, 0.12, 0.005), std=(0.081, 0.32, 0.043), max_pixel_value=255.0, p=1.0),
    ToTensor()
])


In [7]:
transform = albumentations.Compose([
    albumentations.Resize(IMAGENET_SIZE, IMAGENET_SIZE),
    albumentations.OneOf([
        albumentations.RandomGamma(gamma_limit=(60, 120), p=0.9),
        albumentations.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.9),
        albumentations.RandomBrightness(limit=0.2, p=0.9),
        albumentations.RandomContrast(limit=0.2, p=0.9)
        ]),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, interpolation=cv2.INTER_LINEAR,border_mode=cv2.BORDER_CONSTANT, p=1),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightnessContrast(p=0.2),
    albumentations.OneOf([
        albumentations.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
        albumentations.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1)
    ], p=0.5),
    albumentations.Normalize(mean=(0.485, 0.12, 0.005), std=(0.081, 0.32, 0.043), max_pixel_value=255.0, p=1.0),
])

val_transform = albumentations.Compose([
    albumentations.Resize(IMAGENET_SIZE, IMAGENET_SIZE),
    # albumentations.CenterCrop(512, 512),
    # albumentations.JpegCompression(quality_lower=99, quality_upper=100,p=1),
    albumentations.Normalize(mean=(0.485, 0.12, 0.005), std=(0.081, 0.32, 0.043), max_pixel_value=255.0, p=1.0),
])

In [8]:
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, img_labels, transform=None, target_transform=None):
        self.img_labels = img_labels
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx, 0]
        image = Image.open(img_path)
        # image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        # sample = {'image':image, 'label':label}
        if self.transform:
            image_np = np.array(image)
            image = self.transform(image=image_np)['image']
            # image = Image.fromarray(augmented['image'])
        return image, label

In [9]:
Image_size = 224
train_batch_size = 64
val_batch_size = 24
workers = 8

train_dataset = CustomImageDataset(train_df, transform)
val_dataset = CustomImageDataset(val_df, val_transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_batch_size,        
    shuffle=True,
    # sampler=ImbalancedDatasetSampler(train_dataset),
    num_workers=workers,
    pin_memory=True,
    drop_last=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=val_batch_size,        
    shuffle=False,
    num_workers=workers,
    pin_memory=False,
    drop_last=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataloaders = {'train':train_loader,'val':val_loader}

In [10]:
best_acc = 0.0
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_auc = 0.0
    

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.permute(0, 3, 1, 2)
                inputs = inputs.cuda()
                labels = labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print(f'{phase} {dataset_sizes[phase]} {running_corrects} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            # deep copy the model
            global best_acc
            if phase == 'val' and epoch_acc > best_acc:
                for inputs, labels in train_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
model_ft = models.densenet121(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 2)
model = nn.DataParallel(model_ft)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
dataset_sizes = {'train':len(x_train),'val':len(X_test)}
print(dataset_sizes)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=20)

{'train': 20628, 'val': 5158}
Epoch 0/19
----------
train 20628 19164 Loss: 0.1848 Acc: 0.9290
val 5158 4844 Loss: 0.1559 Acc: 0.9391

Epoch 1/19
----------
train 20628 19200 Loss: 0.1796 Acc: 0.9308
val 5158 4840 Loss: 0.1567 Acc: 0.9383

Epoch 2/19
----------
train 20628 19172 Loss: 0.1820 Acc: 0.9294
val 5158 4849 Loss: 0.1562 Acc: 0.9401

Epoch 3/19
----------
train 20628 19215 Loss: 0.1801 Acc: 0.9315
val 5158 4843 Loss: 0.1569 Acc: 0.9389

Epoch 4/19
----------
train 20628 19175 Loss: 0.1825 Acc: 0.9296
val 5158 4846 Loss: 0.1562 Acc: 0.9395

Epoch 5/19
----------
train 20628 19173 Loss: 0.1807 Acc: 0.9295
val 5158 4843 Loss: 0.1563 Acc: 0.9389

Epoch 6/19
----------
train 20628 19183 Loss: 0.1785 Acc: 0.9299
val 5158 4838 Loss: 0.1570 Acc: 0.9380

Epoch 7/19
----------


In [ ]:
torch.save(model_ft,'best_model_1119.pt')

In [23]:
test1_table = pd.read_csv('/data/VPS/VPS_03/lmq/pancreas/data/pancr_net_test1.csv')
test2_table = pd.read_csv('/data/VPS/VPS_03/lmq/pancreas/data/pancr_net_test2.csv')
test1_dataset = CustomImageDataset(test1_table, val_transform)
test2_dataset =  CustomImageDataset(test2_table, val_transform)

test1_loader = torch.utils.data.DataLoader(
    test1_dataset,
    batch_size=1,        
    shuffle=False,
    num_workers=1,
    pin_memory=False,
    drop_last=False)

In [ ]:
test1 = np.array(0,2)
print(test1)